In [2]:
from PIL import Image
from utility import ViltImageSetProcessor
from transformers import ViltImageProcessor, BertTokenizer

In [3]:
img_path = "/home/nikostheodoridis/nuscenes/samples/CAM_BACK/n008-2018-05-21-11-06-59-0400__CAM_BACK__1526915246937570.jpg"

In [4]:
img = Image.open(img_path)

In [9]:
processor = ViltImageProcessor()

In [10]:
x = processor(img)

In [13]:
x["pixel_values"][0].shape

(3, 352, 608)

In [12]:
x

{'pixel_values': [array([[[-0.41176468, -0.41176468, -0.42745095, ..., -0.32549018,
         -0.32549018, -0.31764704],
        [-0.41960782, -0.41176468, -0.41960782, ..., -0.31764704,
         -0.32549018, -0.32549018],
        [-0.42745095, -0.41176468, -0.41176468, ..., -0.3098039 ,
         -0.32549018, -0.3333333 ],
        ...,
        [ 0.5764706 ,  0.5764706 ,  0.58431375, ...,  0.4666667 ,
          0.4431373 ,  0.43529415],
        [ 0.5686275 ,  0.5764706 ,  0.58431375, ...,  0.45882356,
          0.4431373 ,  0.427451  ],
        [ 0.5686275 ,  0.5764706 ,  0.58431375, ...,  0.4666667 ,
          0.45098042,  0.4431373 ]],

       [[-0.38039213, -0.38039213, -0.3960784 , ..., -0.2235294 ,
         -0.2235294 , -0.21568626],
        [-0.38823527, -0.38039213, -0.38823527, ..., -0.21568626,
         -0.2235294 , -0.2235294 ],
        [-0.3960784 , -0.38039213, -0.38039213, ..., -0.20784312,
         -0.2235294 , -0.23137254],
        ...,
        [ 0.56078434,  0.56078434,  